<a href="https://colab.research.google.com/github/josephalanis/thesis_longitudinal_data/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from functools import partial
import math as math
from fractions import Fraction
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
# Creating code to handle Missing data
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.impute import KNNImputer


#Converting a fraction into a p-adic number
%pip install pyadic
from pyadic import PAdic, ModP

In [ ]:
# Set the path to your dataset and Info.txt file

path ="/content/drive/MyDrive/Colab/Thesis/"
info_file ="/content/drive/MyDrive/Colab/Thesis/Miss.csv"
info_file1="/content/drive/MyDrive/Colab/Thesis/dtTrail3.csv"
info_file2="/content/drive/MyDrive/Colab/Thesis/s2.csv"

Miss = pd.read_csv(info_file)
Original=pd.read_csv(info_file1)
sample=pd.read_csv(info_file2)
#info = info.drop('Unnamed: 7', axis=1)



In [ ]:
#Viewing Dataframe
Miss
print(sample.head())
print(Original.head())



      id    Age  Sex  Trt  Error     BP0     BP1     BP2
0   9939  50.97    1    0  -1.12  152.78  148.25  149.36
1  33905  52.09    0    1  -2.43  145.69  146.69  140.55
2   7420  46.86    0    1   1.89  144.52  143.81  139.30
3  13429  53.73    1    0   2.20  162.06  158.48  156.92
4  65089  48.39    0    0  -1.43  141.41     NaN     NaN
   id  Age  Sex  Trt  Error  BP0  BP1  BP2  index
0   1   52    1    0     -2  157  155  151      1
1   2   53    1    0      0  162  159  160      2
2   3   49    0    0      2  149  145  144      3
3   4   47    1    0     -3  140  137  134      4
4   5   53    1    0      1  160  158  155      5


In [ ]:
# Code for missing data
import numpy as np


class Corruptor:

    def __init__(self, data, dtype=np.float64):
        self.dtype = data.dtype
        self.shape = np.shape(data)
        self.data = data.astype(dtype)

    def mcar(self):
        """ Overwrites values with MCAR placed NaN's """
        data_1d = self.flatten()
        n_total = len(data_1d)
        nan_x = np.random.choice(range(n_total),
                                  size=int(n_total*0.3),
                                  replace=False)
        for x_i in nan_x:
            data_1d[x_i] = np.nan
        output = data_1d.reshape(self.shape)
        return output


In [ ]:
class Corruptor1:

    def __init__(self, data, dtype=np.float64):
        self.dtype = data.dtype
        self.shape = np.shape(data)
        self.data = data.astype(dtype)

    def mcar1(self):
        """ Overwrites values with MCAR placed NaN's """
        data_1d1 = self.flatten()
        n_total = len(data_1d1)
        nan_x1 = np.random.choice(range(n_total),
                                  size=int(n_total*0.5),
                                  replace=False)

        for x_i1 in nan_x1:
            data_1d1[x_i1] = np.nan
        output1 = data_1d1.reshape(self.shape)
        return output1


In [ ]:
#need to check why corruptor1 is not working
sp=Original.sample(n=200)
sc1=np.array([sp['BP2']])
sc1=sc1.astype(float)
print(sc1)
m1=Corruptor1.mcar1(sc1)
m1=m1.flatten()
print(m1)

[[150. 141. 150. 152. 147. 148. 147. 151. 140. 148. 159. 137. 159. 151.
  158. 141. 136. 144. 141. 146. 133. 153. 166. 153. 157. 138. 159. 132.
  147. 156. 134. 142. 133. 173. 141. 150. 138. 138. 142. 159. 157. 150.
  142. 152. 142. 150. 138. 142. 140. 144. 143. 145. 146. 145. 165. 135.
  142. 147. 167. 147. 144. 144. 137. 156. 136. 151. 149. 150. 139. 160.
  138. 167. 144. 154. 142. 161. 146. 144. 149. 175. 154. 155. 153. 137.
  172. 165. 156. 139. 140. 149. 173. 139. 156. 160. 149. 135. 144. 138.
  183. 142. 155. 146. 164. 147. 144. 157. 166. 152. 156. 146. 141. 152.
  149. 162. 171. 143. 184. 165. 140. 147. 156. 133. 135. 141. 134. 149.
  149. 145. 144. 164. 137. 137. 158. 151. 171. 148. 150. 157. 142. 141.
  147. 141. 158. 171. 150. 177. 135. 146. 139. 161. 165. 159. 160. 146.
  150. 146. 139. 169. 155. 147. 166. 142. 163. 136. 151. 154. 141. 137.
  145. 132. 141. 143. 155. 133. 157. 168. 146. 182. 144. 149. 152. 145.
  138. 154. 164. 167. 148. 147. 147. 138. 149. 149. 165. 142. 15

In [ ]:
#Creating code to find bias
def bias(X,Y):
    diff=Y-X
    m=diff.mean()
    return m

In [ ]:
#Calculating RMSE
from sklearn.metrics import mean_squared_error

In [ ]:
#creating for loop to find everything
#empyt
#b1 = np.zeros((100, 1))
#b2=np.zeros((100, 1))
#impb1=np.zeros((100, 1))
#impb2=np.zeros((100, 1))

imputer=KNNImputer(n_neighbors=3,weights='uniform',metric='nan_euclidean')
nan=float("NaN")

#results=np.zeros((100,4))

def thesis(X):
  results=np.zeros((500,4))
  num_test=results.shape[0]
  num_train=results.shape[1]
  for i in range(num_test):
    #for j in range(4):
      #Creating random sample of 100
      sp=Original.sample(n=1000)
      #print("Original", sp)
      #removing the columns I don't need and creating missing data:
      sc=np.array([sp['BP1']])
      sc=sc.astype(float)
      sc1=np.array([sp['BP2']])
      sc1=sc1.astype(float)
      m=Corruptor.mcar(sc)
      m=m.flatten()
      m1=Corruptor1.mcar1(sc1)
      m1=m1.flatten()
      #print(m1)
      #Need to combine these into one data frame with BP0,m,m1
      sd=np.array(sp['BP0'])
      sd=sd.astype(float)
      #Combined as an array
      arr = np.column_stack((sd, m,m1))
      #Combined as a dataframe
      imp=pd.DataFrame({'BP0':arr[:,0],'BP1':arr[:,1],'BP2':arr[:,2]})
      #print(imp)
      #Now try the imputer on the data frame of BP0 - BP1
      sp_imputed=np.round(imputer.fit_transform(imp),8)
      #print(sp_imputed.shape)
      sp_imputed=pd.DataFrame({'BP0':sp_imputed[:,0],'BP1':sp_imputed[:,1],'BP2':sp_imputed[:,2]})
      #print("imputed", sp_imputed)

       #Find the  bias
      b1=bias(sp['BP1'],sp_imputed['BP1'])
      b2=bias(sp['BP2'],sp_imputed['BP2'])
      type(b1)==float
      type(b2)==float
      b1=round(b1,4)
      b2=round(b2,4)
      #print(b1)
      #print(b2)
      #Find RMSE
      r1 = mean_squared_error(sp['BP1'],sp_imputed['BP1'], squared=False)
      type(r1)==float
      r1=round(r1,4)
      r2 = mean_squared_error(sp['BP2'],sp_imputed['BP2'], squared=False)
      type(r2)==float
      r2=round(r2,4)
      #Storing Results
      results[i,0]=b1
      results[i,1]=b2
      results[i,2]=r1
      results[i,3]=r2
  return results



In [ ]:
Y=thesis(Original)
#print(Y.shape)
#print(Y)

In [ ]:
impdata=pd.DataFrame({'BP1_Bias':Y[:,0],'BP2_Bias':Y[:,1], 'BP1_RMSE':Y[:,2], 'BP2_RMSE':Y[:,3] })
#print(impdata.head())

In [ ]:
final1=impdata.fillna(0)
#print(final1)
final2=final1.mean()
print(final2.round(4))

BP1_Bias   -0.1136
BP2_Bias   -0.2192
BP1_RMSE    1.3049
BP2_RMSE    1.7297
dtype: float64


#creating sample of 500
#Creating random sample of 100
sp=Original.sample(n=100)

#print(sp.head())
#removing the columns I don't need and creating missing data:
sc=np.array([sp['BP1']])
sc=sc.astype(float)
sc1=np.array([sp['BP2']])
sc1=sc1.astype(float)
m=Corruptor.mcar(sc)
m=m.flatten()
m1=Corruptor.mcar(sc1)
m1=m1.flatten()


#Need to combine these into one data frame with BP0,m,m1
sd=np.array(sp['BP0'])
sd=sd.astype(float)
#Combined as an array
arr = np.column_stack((sd, m,m1))
#Combined as a dataframe
imp=pd.DataFrame({'BP0':arr[:,0],'BP1':arr[:,1],'BP2':arr[:,2]})

print(imp.head)



In [ ]:
# Creating code to handle Missing data
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.impute import KNNImputer

imputer=KNNImputer(n_neighbors=3,weights='uniform',metric='nan_euclidean')
imputer1=KNNImputer(n_neighbors=3,weights='distance',metric='nan_euclidean')
nan=float("NaN")


In [ ]:
#Now try the imputer on the data frame of BP0 - BP1
sp_imputed=np.round(imputer.fit_transform(imp),2)
#print(sp_imputed)

sp_imputed=pd.DataFrame({'BP0':sp_imputed[:,0],'BP1':sp_imputed[:,1],'BP2':sp_imputed[:,2]})
#sp_imputed1=np.round(imputer.fit_transform(imp),2)
print(sp_imputed.head)
print(sp_imputed.var())

<bound method NDFrame.head of        BP0     BP1     BP2
0    171.0  166.00  165.00
1    160.0  164.00  154.00
2    146.0  145.00  140.00
3    146.0  143.00  145.00
4    165.0  162.00  161.33
..     ...     ...     ...
495  145.0  141.33  141.00
496  140.0  137.00  135.00
497  144.0  142.00  140.00
498  177.0  176.33  174.00
499  161.0  160.00  156.00

[500 rows x 3 columns]>
BP0    112.430846
BP1    111.619025
BP2    115.482222
dtype: float64


In [ ]:
#Calculating RMSE and Bias
from sklearn.metrics import mean_squared_error
#converting imputed datat into a dataframe
#sp1=pd.DataFrame('BP0':sp_imputed[:,0],'BP1':sp_imputed[:,0])

#sp1=np.array(sp_imputed['BP1'])
r1 = mean_squared_error(sp['BP1'], sp_imputed['BP1'], squared=False)
print(r1)
r2=mean_squared_error(sp['BP2'], sp_imputed['BP2'], squared=False)
print(r2)

0.6037444823764443
0.7151041882131581


In [ ]:
#Converting dataframe to nparray
M=Miss.to_numpy()
print(M.shape)

#Transposing Missing data array
M=M.reshape(M.shape[1],-1)
print(M.shape)

#Creating numpy array to check distance with
list1=np.random.rand(1,100)
print(list1.shape)

(100, 1)
(1, 100)
(1, 100)


In [ ]:
#Creating set for KNN Model
from sklearn.model_selection import train_test_split
import pandas as pd

# Keep last row of each subject
subjects = sample.groupby('id').last().reset_index()

# Split this data stratifying by `trt`
x_train, x_test, y_train, y_test = train_test_split(subjects['BP1'],subjects["Trt"], train_size=0.8, test_size=0.2, stratify=subjects['Trt'],random_state=42)

In [ ]:
print(y_train.shape)
print(x_train.shape)
print(y_test.shape)
print(x_test.shape)

#converting x_test and x_train to numpy array
x_train=np.array([x_train])
x_test=np.array([x_test])

print(x_train)
print(x_test)

print(x_train.shape)
print(x_test.shape)


(80,)
(80,)
(20,)
(20,)
[[158.07 158.93 148.69 161.02 145.15 147.41 143.35 153.52 162.42 163.37
  158.48 151.91 160.87 142.77 150.55 146.4  165.93 166.57    nan 150.67
  158.5  153.22 137.54    nan 138.5  135.29 142.57 146.06 161.13 150.73
  144.55    nan 161.4  145.41 178.57    nan 142.5  141.88 167.73 151.24
  160.26 150.91 155.99 140.88 149.99 170.8  158.28 142.83 146.65 149.9
     nan 173.46 148.79 150.   148.22 138.8  140.41 148.97 146.69 142.67
  139.95 144.31    nan 143.81    nan 141.84 156.55    nan    nan 177.28
  148.76 160.37 145.38 148.25    nan 168.23 159.71 162.39 148.45 151.48]]
[[165.9  155.05 158.67 142.8  149.07 167.62 163.59 140.5  144.6  177.93
  165.68 145.88 161.46 150.52 157.65 146.59 147.66 144.85 149.97 149.71]]
(1, 80)
(1, 20)


In [ ]:
#Code Need for P-adic distances

#Checking Prime Number
def create_seive(n):
    '''Creates a list of primality of integers up to n.
    n Should be an positive, if not integer will round down to closest int '''
    try:
        n = int(n)
    except ValueError:
        raise ValueError

    assert n > 0

    my_list = [True] * (n+1)
    my_list[0] = False
    my_list[1] = False

    for j in range(2, n//2 + 1):
        for i in range(j, n+1, j):
            if i != j:
                my_list[i] = False

    return my_list

max_prime = 1000
is_prime_list = create_seive(max_prime)

def prime_check(p):
    '''Given a p check that it is a prime number between 0 and 1000'''
    try:
        p = int(p)
        is_prime = is_prime_list[p]
    except ValueError:
        return False
    return is_prime

def validate_prime(p):
    if not prime_check(p):
        raise ValueError

def find_p_ord_int(a, p):
    '''Calculate the ordinal of a, an int, with respect to p a prime.'''
    if a == 0:
        validate_prime(p)
        return float("inf")
    elif isinstance(a, Fraction):
        raise 'Error a is a fraction must be an int.'
    try:
        a = int(a)
    except ValueError:
        raise ValueError
    validate_prime(p)
    c = 0
    while a % (p**c) == 0:
        c += 1
    return c - 1

def find_p_ord(a, p):
    '''Calculate the ordinal of a rational number a
        with respect to a prime p.'''
    if a == 0:
        validate_prime(p)
        return float(0)
    elif isinstance(a, int):
        return find_p_ord_int(a, p)
    else:
        try:
            frac = Fraction(str(a)).limit_denominator()
        except ValueError:
            raise ValueError
        numerator = find_p_ord_int(frac.numerator, p)
        denominator = find_p_ord_int(frac.denominator, p)
        return numerator - denominator

#    '''Calculate the p-adic norm of a, a rational number,with repect to p a prime.''
def p_norm(self, p):
    vect=self.shape[0]
    values=np.zeros(vect)

    for k in range(vect):
        a=self[k].flatten().max()
        values[k]=1/(p**find_p_ord(a, p))
    return values

In [ ]:
# First creating euclidean code  for self=(1,3) and X=(1,3)
def na_euclidean(X, Y):
    num_test = X.shape[0]
    num_train = Y.shape[0]
    dists = np.zeros((num_test, num_train))
    dists=nan_euclidean_distances(X,Y)
    return dists

In [ ]:
u=na_euclidean(list1,M)
print(u)

[[1516.49588097]]


In [ ]:
# Seconding creating hammington distance for missing data
from sklearn.metrics.pairwise import manhattan_distances
import itertools
import warnings
from functools import partial
from numbers import Integral, Real

import numpy as np
from joblib import effective_n_jobs
from scipy.sparse import csr_matrix, issparse
from scipy.spatial import distance

from sklearn import config_context
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import normalize
from sklearn.utils import (
    check_array,
    gen_batches,
    gen_even_slices,
    get_chunk_n_rows,
    is_scalar_nan,
)
from sklearn.utils._mask import _get_mask

from sklearn.utils.extmath import row_norms, safe_sparse_dot

from sklearn.utils.parallel import Parallel, delayed
from sklearn.utils.validation import _num_samples, check_non_negative

#need this code
def _return_float_dtype(X, Y):
    """
    1. If dtype of X and Y is float32, then dtype float32 is returned.
    2. Else dtype float is returned.
    """
    if not issparse(X) and not isinstance(X, np.ndarray):
        X = np.asarray(X)

    if Y is None:
        Y_dtype = X.dtype
    elif not issparse(Y) and not isinstance(Y, np.ndarray):
        Y = np.asarray(Y)
        Y_dtype = Y.dtype
    else:
        Y_dtype = Y.dtype

    if X.dtype == Y_dtype == np.float32:
        dtype = np.float32
    else:
        dtype = float

    return X, Y, dtype


#Need this code
def check_pairwise_arrays(
    X,
    Y,
    *,
    precomputed=False,
    dtype=None,
    accept_sparse="csr",
    force_all_finite=True,
    copy=False,):

    X, Y, dtype_float = _return_float_dtype(X, Y)

    estimator = "check_pairwise_arrays"
    if dtype is None:
        dtype = dtype_float

    if Y is X or Y is None:
        X = Y = check_array(
            X,
            accept_sparse=accept_sparse,
            dtype=dtype,
            copy=copy,
            force_all_finite=force_all_finite,
            estimator=estimator,
        )
    else:
        X = check_array(
            X,
            accept_sparse=accept_sparse,
            dtype=dtype,
            copy=copy,
            force_all_finite=force_all_finite,
            estimator=estimator,
        )
        Y = check_array(
            Y,
            accept_sparse=accept_sparse,
            dtype=dtype,
            copy=copy,
            force_all_finite=force_all_finite,
            estimator=estimator,
        )

    if precomputed:
        if X.shape[1] != Y.shape[0]:
            raise ValueError(
                "Precomputed metric requires shape "
                "(n_queries, n_indexed). Got (%d, %d) "
                "for %d indexed." % (X.shape[0], X.shape[1], Y.shape[0])
            )
    elif X.shape[1] != Y.shape[1]:
        raise ValueError(
            "Incompatible dimension for X and Y matrices: "
            "X.shape[1] == %d while Y.shape[1] == %d" % (X.shape[1], Y.shape[1])
        )

    return X, Y


def nan_manhattan_distances(X, Y=None, missing_values=np.nan, copy=True):

    force_all_finite = "allow-nan" if is_scalar_nan(missing_values) else True
    X, Y = check_pairwise_arrays(
        X, Y, accept_sparse=False, force_all_finite=force_all_finite, copy=copy)

    # Get missing mask for X
    missing_X = _get_mask(X, missing_values)

    # Get missing mask for Y
    missing_Y = missing_X if Y is X else _get_mask(Y, missing_values)

    # set missing values to zero
    X[missing_X] = 0
    Y[missing_Y] = 0

    X, Y = check_pairwise_arrays(X, Y)

    distances = manhattan_distances(X, Y)

    return distances

In [ ]:
#Checking if nan_manhattan_distances code works
h=nan_manhattan_distances(list1,M)
print(h)

[[12117.48587576]]


In [ ]:
# now creating p-adic distances with missing data using manhattan distances
def nan_padic_distances(X, Y,p):
    num_test = X.shape[0]
    num_train = Y.shape[0]
    dists = np.zeros((num_test, num_train))
    dist=nan_manhattan_distances(X,Y)
    dists = p_norm(dist,p)
    dists=dists.reshape(-1,1)
    return dists


In [ ]:
t=nan_padic_distances(list1,M,5)
print(t)

[[1.]]


In [ ]:
# List of distance metrics to try
distance_metrics = ['exc_dist','nan_euclidean','nan_manhattan']
prime_numbers = [2,3,5,7,11,13,17,19,23,29]
n_neighbors = 3


#print(x_train.shape)
#print(y_train.shape)

In [ ]:
#Using KNN Imputer to fill in missing data
from sklearn.impute import KNNImputer

imputer=KNNImputer(n_neighbors=3,weights='uniform',metric='nan_euclidean')

print(x_train)
print(x_test)
x_train=imputer.fit_transform(x_train)
x_test=imputer.fit_transform(x_test)
print(x_train)
print(x_test)

print(y_train.shape)
print(x_train.shape)



[[158.07 158.93 148.69 161.02 145.15 147.41 143.35 153.52 162.42 163.37
  158.48 151.91 160.87 142.77 150.55 146.4  165.93 166.57 150.67 158.5
  153.22 137.54 138.5  135.29 142.57 146.06 161.13 150.73 144.55 161.4
  145.41 178.57 142.5  141.88 167.73 151.24 160.26 150.91 155.99 140.88
  149.99 170.8  158.28 142.83 146.65 149.9  173.46 148.79 150.   148.22
  138.8  140.41 148.97 146.69 142.67 139.95 144.31 143.81 141.84 156.55
  177.28 148.76 160.37 145.38 148.25 168.23 159.71 162.39 148.45 151.48]]
[[165.9  155.05 158.67 142.8  149.07 167.62 163.59 140.5  144.6  177.93
  165.68 145.88 161.46 150.52 157.65 146.59 147.66 144.85 149.97 149.71]]
[[158.07 158.93 148.69 161.02 145.15 147.41 143.35 153.52 162.42 163.37
  158.48 151.91 160.87 142.77 150.55 146.4  165.93 166.57 150.67 158.5
  153.22 137.54 138.5  135.29 142.57 146.06 161.13 150.73 144.55 161.4
  145.41 178.57 142.5  141.88 167.73 151.24 160.26 150.91 155.99 140.88
  149.99 170.8  158.28 142.83 146.65 149.9  173.46 148.79 150.  

In [ ]:
#Now try the imputer on the data frame of BP0 - BP1
#print(sample)
sample1=pd.DataFrame(sample[['BP0','BP1','BP2']])
print(sample1)
sample_imputed=np.round(imputer.fit_transform(sample1),2)
print(sample_imputed)

#Check Bias


       BP0     BP1     BP2
0   152.78  148.25  149.36
1   145.69  146.69  140.55
2   144.52  143.81  139.30
3   162.06  158.48  156.92
4   141.41     NaN     NaN
..     ...     ...     ...
95  146.70  144.60  142.53
96  147.88  142.50  143.44
97  171.22  165.90  168.85
98  156.44  155.99  150.91
99  150.28  147.66  148.13

[100 rows x 3 columns]
[[152.78 148.25 149.36]
 [145.69 146.69 140.55]
 [144.52 143.81 139.3 ]
 [162.06 158.48 156.92]
 [141.41 138.24 140.6 ]
 [148.49 148.76 143.8 ]
 [162.1  161.46 155.78]
 [143.69 138.5  141.81]
 [148.38 146.65 141.03]
 [145.84 142.83 143.87]
 [139.32 138.24 140.6 ]
 [148.69 148.45 140.39]
 [149.74 146.06 145.77]
 [144.98 144.55 141.74]
 [141.98 135.29 141.81]
 [143.77 140.41 141.28]
 [168.32 163.59 165.42]
 [142.42 139.95 141.28]
 [150.63 150.73 144.49]
 [160.22 158.67 157.56]
 [157.19 160.26 150.18]
 [156.9  150.67 151.73]
 [156.18 153.22 152.47]
 [151.95 149.99 148.05]
 [163.38 161.4  161.35]
 [165.86 160.37 160.78]
 [171.07 168.23 163.38]
 [14

In [ ]:
#Calculating RMSE and Bias
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_actual, y_predicted, squared=False)
